In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataaaa/train_trial1.csv


In [1]:
import pandas as pd

In [3]:
data = pd.read_csv('/kaggle/input/dataaaa/train_trial1.csv')

In [4]:
data.dropna(subset=['captions'],inplace=True)

In [5]:
l = []
for i in data['likes']:
    if "," not in i:
        l.append(int(i))
    else:
        a = i.split(",")
        l.append(int("".join(a)))
data['likes'] = l

In [7]:
# Define neural network architecture with classification head
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from transformers import BertTokenizer, BertModel

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
class CaptionLikesPredictor(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(CaptionLikesPredictor, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(768, num_classes)  # Output dimension is number of classes

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        class_probs = self.fc(pooled_output)
        return class_probs

# Prepare data
# Assuming you have a DataFrame called 'data' with columns 'caption' and 'likes'
# Identify the most liked caption for each sample
# Identify the most liked caption for each sample
most_liked_caption_indices = data.groupby('Unnamed: 0')['likes'].idxmax()
data['is_most_liked'] = 0
data.loc[most_liked_caption_indices, 'is_most_liked'] = 1


# Split data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Tokenize captions
def tokenize_text(text):
    return tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

train_data['tokenized'] = train_data['captions'].apply(tokenize_text)
test_data['tokenized'] = test_data['captions'].apply(tokenize_text)

# Convert DataFrame to PyTorch Dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data.iloc[index]
        input_ids = item['tokenized']['input_ids'].squeeze(0)
        attention_mask = item['tokenized']['attention_mask'].squeeze(0)
        is_most_liked = torch.tensor(item['is_most_liked'], dtype=torch.long)
        return input_ids, attention_mask, is_most_liked

train_dataset = CustomDataset(train_data)
test_dataset = CustomDataset(test_data)

# Define DataLoader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize model
model = CaptionLikesPredictor(bert_model, num_classes=2)  # 2 classes: most liked or not

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for input_ids, attention_mask, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        test_losses = []
        for input_ids, attention_mask, labels in test_loader:
            outputs = model(input_ids, attention_mask)
            test_loss = criterion(outputs, labels)
            test_losses.append(test_loss.item())

    print(f'Epoch {epoch+1}/{num_epochs}, Test Loss: {np.mean(test_losses)}')


OSError: Can't load tokenizer for 'bert-base-uncased'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'bert-base-uncased' is the correct path to a directory containing all relevant files for a BertTokenizer tokenizer.

In [9]:
torch.save(model,'/kaggle/working/modelcap.pth')

In [8]:
def prepare_caption(text):
    return tokenizer.encode_plus(text, add_special_tokens=True, max_length=128, truncation=True, return_tensors='pt')

# Define a function to predict probabilities for each caption
def predict_probability(model, tokenized_caption):
    with torch.no_grad():
        input_ids = tokenized_caption['input_ids'].squeeze(0)
        attention_mask = tokenized_caption['attention_mask'].squeeze(0)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probabilities = torch.softmax(outputs.logits, dim=1)[0][1].item()  # Assuming 1 is the index for 'most liked'
    
    return probabilities

In [35]:
df = pd.read_csv('/kaggle/input/captiondataset/final_caps.csv')

In [36]:
df.head()

,family trip,friends trip,abstract art,meeting,wedding,sunset,fitness,scenery
0,My family will always travel together,I’ve found my happy place,Spot the CEO,"“Right now, I am confused about every single t...",Chasing daylight,"A new morning, a new beginning",© 2024 Later,Living in the rhythm of nature
1,Seeing your kids smile makes all the stress an...,What’s on my bucket list? Everywhere,'The greatest glory in living lies not in neve...,Better than Pretzel Day,Hues of tranquillity,All of the letdowns of yesterday are redeemed ...,© 2024 Later,Impressions of untouched wilderness
2,I would travel my entire life if I could alway...,"Take only pictures, leave only footprints",My favorite exercise is a cross between a lung...,"""I never doubted you were mine",Fiery farewell,"And in the middle of waiting for sunrise, I fo...",This Barbie got her workout in,The dance of the sea meeting the sky
3,I’ll keep these memories in my heart until our...,Always say yes to new adventures,Patience — what you have when there are too ma...,"“If you care about someone, and you got a litt...",Cotton candy skies,Another sunrise granted by the almighty,Just peachy,Mountain melodies whispering in the wind
4,"No matter where we go in the world, we will al...",Life is short and the world is wide,Don’t give up on your dreams,“Marry me… because I'd like to date you,Sunkissed evenings,"Best kind of gold! Plus, it can’t be sold",Omg I'm hot girl fit,Views that leave you speechless


In [37]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Define a function to tokenize and prepare captions
def prepare_caption(text):
    return tokenizer.encode_plus(text, add_special_tokens=True, max_length=128, truncation=True, return_tensors='pt')

# Example set of captions
captions = list(df['family trip'])

# Tokenize and prepare captions
tokenized_captions = [prepare_caption(caption) for caption in captions]

# Predict probabilities for each caption
# Predict probabilities for each caption
def predict_probabilities(model, tokenized_captions):
    probabilities = []
    with torch.no_grad():
        for tokenized_caption in tokenized_captions:
            input_ids = tokenized_caption['input_ids'].squeeze(0)
            attention_mask = tokenized_caption['attention_mask'].squeeze(0)

            # Pad or truncate input_ids to a fixed length
            max_length = 128
            input_ids = torch.cat([input_ids, torch.zeros(max_length - input_ids.shape[0], dtype=torch.long)])
            attention_mask = torch.cat([attention_mask, torch.zeros(max_length - attention_mask.shape[0], dtype=torch.long)])

            # Ensure input_ids and attention_mask have correct shape
            input_ids = input_ids.unsqueeze(0)
            attention_mask = attention_mask.unsqueeze(0)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            probabilities.append(torch.softmax(outputs, dim=1)[0][1].item())  # Assuming 1 is the index for 'most liked'
    return probabilities


# Load your trained model
# loaded_model = YourLoadedModelClass(bert_model, num_classes=2)  # Example

# Predict probabilities
predicted_probabilities = predict_probabilities(model, tokenized_captions)

# Display results
results = pd.DataFrame({'Caption': captions, 'Probability': predicted_probabilities})
print(results)


                                               Caption  Probability
0                My family will always travel together     0.993031
1    Seeing your kids smile makes all the stress an...     0.992589
2    I would travel my entire life if I could alway...     0.992488
3    I’ll keep these memories in my heart until our...     0.991265
4    No matter where we go in the world, we will al...     0.992127
..                                                 ...          ...
195                  Life is far too short for routine     0.993283
196                Wasted time here is time well spent     0.993320
197     I haven't been everywhere, but it's on my list     0.993493
198                          I collect passport stamps     0.992503
199  Whenever I regret the money I spent traveling,...     0.991715

[200 rows x 2 columns]


In [38]:
results = results.sort_values(by='Probability',ascending=False)

In [39]:
results

,Caption,Probability
165,"Exploring hearts, exploring places",0.995198
164,Travel and togetherness,0.994702
120,"Fun and laughter, all day, every day! <img al...",0.994618
127,"Exploring new horizons, hand in hand",0.994581
147,"Family and travel, a perfect blend",0.994547
...,...,...
83,“Don’t just tell your children about the world,0.988519
177,“Don’t just tell your children about the world,0.988519
97,“Having kids is a reason to travel,0.987365
188,“Don’t count the days,0.987057


In [42]:
new_df = pd.DataFrame()

In [43]:
new_df['family trip'] = results['Caption']

In [44]:
new_df

,family trip
165,"Exploring hearts, exploring places"
164,Travel and togetherness
120,"Fun and laughter, all day, every day! <img al..."
127,"Exploring new horizons, hand in hand"
147,"Family and travel, a perfect blend"
...,...
83,“Don’t just tell your children about the world
177,“Don’t just tell your children about the world
97,“Having kids is a reason to travel
188,“Don’t count the days


In [29]:
df.columns

Index(['family trip', 'friends trip', 'abstract art', 'meeting', 'wedding',
       'sunset', 'fitness', 'scenery'],
      dtype='object')

In [45]:
captions = list(df['friends trip'])
tokenized_captions = [prepare_caption(caption) for caption in captions]
predicted_probabilities = predict_probabilities(model, tokenized_captions)

# Display results
results = pd.DataFrame({'Caption': captions, 'Probability': predicted_probabilities})
results = results.sort_values(by='Probability',ascending=False)
new_df['friends trip']=results['Caption']

In [46]:
new_df

,family trip,friends trip
165,"Exploring hearts, exploring places","“Because of you, I laugh a little louder and s..."
164,Travel and togetherness,“Friendship consists in forgetting what one gi...
120,"Fun and laughter, all day, every day! <img al...",“A day spent with friends is always a day well...
127,"Exploring new horizons, hand in hand","“With friends, every day is a celebration"
147,"Family and travel, a perfect blend",“A true friend is one soul in two bodies
...,...,...
83,“Don’t just tell your children about the world,“We’re the ‘let’s just wing it’ specialists
177,“Don’t just tell your children about the world,"“When worst comes to worst, squad comes first"
97,“Having kids is a reason to travel,“Forever grateful for these moments
188,“Don’t count the days,"“Sisters at heart, partners in crime"


In [47]:
captions = list(df['abstract art'])
tokenized_captions = [prepare_caption(caption) for caption in captions]
predicted_probabilities = predict_probabilities(model, tokenized_captions)

# Display results
results = pd.DataFrame({'Caption': captions, 'Probability': predicted_probabilities})
results = results.sort_values(by='Probability',ascending=False)
new_df['abstract art']=results['Caption']

In [48]:
captions = list(df['meeting'])
tokenized_captions = [prepare_caption(caption) for caption in captions]
predicted_probabilities = predict_probabilities(model, tokenized_captions)

# Display results
results = pd.DataFrame({'Caption': captions, 'Probability': predicted_probabilities})
results = results.sort_values(by='Probability',ascending=False)
new_df['meeting']=results['Caption']

In [49]:
captions = list(df['wedding'])
tokenized_captions = [prepare_caption(caption) for caption in captions]
predicted_probabilities = predict_probabilities(model, tokenized_captions)

# Display results
results = pd.DataFrame({'Caption': captions, 'Probability': predicted_probabilities})
results = results.sort_values(by='Probability',ascending=False)
new_df['wedding']=results['Caption']

In [50]:
captions = list(df['sunset'])
tokenized_captions = [prepare_caption(caption) for caption in captions]
predicted_probabilities = predict_probabilities(model, tokenized_captions)

# Display results
results = pd.DataFrame({'Caption': captions, 'Probability': predicted_probabilities})
results = results.sort_values(by='Probability',ascending=False)
new_df['sunset']=results['Caption']

In [51]:
captions = list(df['fitness'])
tokenized_captions = [prepare_caption(caption) for caption in captions]
predicted_probabilities = predict_probabilities(model, tokenized_captions)

# Display results
results = pd.DataFrame({'Caption': captions, 'Probability': predicted_probabilities})
results = results.sort_values(by='Probability',ascending=False)
new_df['fitness']=results['Caption']

In [52]:
captions = list(df['scenery'])
tokenized_captions = [prepare_caption(caption) for caption in captions]
predicted_probabilities = predict_probabilities(model, tokenized_captions)

# Display results
results = pd.DataFrame({'Caption': captions, 'Probability': predicted_probabilities})
results = results.sort_values(by='Probability',ascending=False)
new_df['scenery']=results['Caption']

In [54]:
new_df.to_csv('/kaggle/working/sorted_captions.csv',index=False)

In [14]:
for col in df.columns:
    df[f'{col}_tokenized'] = df[col].apply(prepare_caption)

# Predict probabilities for each caption in each column
for col in df.columns:
    if '_tokenized' in col:
        df[f'{col}_probability'] = df[col].apply(lambda x: predict_probability(model, x))

# Sort each column based on the predicted probabilities in descending order
for col in df.columns:
    if '_probability' in col:
        df = df.sort_values(by=col, ascending=False)

# Drop intermediate tokenized columns
df = df.drop(columns=[col for col in df.columns if '_tokenized' in col])

# Print or use sorted data as needed
print(df)

ValueError: Input {'input_ids': tensor([[ 101, 2026, 2155, 2097, 2467, 3604, 2362,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])} is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.